In [2]:
import numpy as np
import pandas as pd

import csv

# Open the TSV file in read mode with 't' flag for text mode
with open(
    "/Users/lujun.li/projects/causallm-github/causalllm/resource/input/ML/floresp-v2.0-rc.2/metadata_devtest.tsv",
    "rt",
) as tsvfile:
    # Create a CSV reader object with tab delimiter
    reader = csv.reader(tsvfile, delimiter="\t")
    # Iterate through each row in the TSV file
    for row in reader:
        # Do something with each row, for example, print it
        print(row)

['URL', 'domain', 'topic', 'has_image', 'has_hyperlink']
['https://en.wikinews.org/wiki/Toronto_team-led_research_on_Type_1_Diabetes_%27groundbreaking%27', 'wikinews', 'disease, research, canada', 'no', 'yes']
['https://en.wikinews.org/wiki/Toronto_team-led_research_on_Type_1_Diabetes_%27groundbreaking%27', 'wikinews', 'disease, research, canada', 'no', 'yes']
['https://en.wikinews.org/wiki/Toronto_team-led_research_on_Type_1_Diabetes_%27groundbreaking%27', 'wikinews', 'disease, research, canada', 'no', 'yes']
['https://en.wikinews.org/wiki/Nobel_Prize_in_Literature_Committee_abandons_efforts_to_contact_Bob_Dylan', 'wikinews', 'music', 'yes', 'yes']
['https://en.wikinews.org/wiki/Nobel_Prize_in_Literature_Committee_abandons_efforts_to_contact_Bob_Dylan', 'wikinews', 'music', 'yes', 'yes']
['https://en.wikinews.org/wiki/Amazon_to_buy_smart_doorbell_startup_Ring', 'wikinews', 'business', 'no', 'yes']
['https://en.wikinews.org/wiki/Amazon_to_buy_smart_doorbell_startup_Ring', 'wikinews', '

In [3]:
from datasets import load_dataset

datasets = {}

configs = ["eng_Latn", "mlt_Latn", "ind_Latn", "lvs_Latn", "isl_Latn", "khm_Khmr"]

code_to_language_dict = {
    "eng_Latn": "english",
    "mlt_Latn": "maltese",
    "ind_Latn": "indonesian",
    "lvs_Latn": "standard latvian",
    "isl_Latn": "icelandic",
    "khm_Khmr": "khmer",
}

for config in configs:
    datasets[config] = load_dataset("facebook/flores", config)

for config, dataset in datasets.items():
    dataset.save_to_disk(f"resource/input/datasets/{config}")

/opt/anaconda3/envs/causalLLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/causalLLM/lib/python3.9/site-packages/datasets/load.py:1486: FutureWarning: The repository for facebook/flores contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/flores
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 1012/1012 [00:00<00:00, 113145.03 examples/s]


In [4]:
english_ds = load_dataset(f"resource/input/datasets/eng_Latn", split="validation")
mlt_ds = load_dataset(f"resource/input/datasets/mlt_Latn", split="validation")

Generating validation split: 1 examples [00:00, 198.58 examples/s]
Generating validation split: 1 examples [00:00, 581.73 examples/s]


In [5]:
dataframes = {}
for dataset in datasets:
    df = pd.DataFrame(datasets[dataset]["devtest"])
    dataframes[dataset] = df
    dataframes[dataset].to_parquet(
        f"resource/input/datasets/{dataset}.parquet", index=False
    )

In [6]:
eng_ds = pd.read_parquet(
    "/Users/lujun.li/projects/causallm-github/causalllm/resource/input/ML/datasets/eng_Latn.parquet"
)

In [7]:
mlt_Latn_ds = pd.read_parquet(
    "/Users/lujun.li/projects/causallm-github/causalllm/resource/input/ML/datasets/mlt_Latn.parquet"
)

In [8]:
merged_df_list = []
eng_ds = pd.read_parquet(
    "/Users/lujun.li/projects/causallm-github/causalllm/resource/input/ML/datasets/eng_Latn.parquet"
)
for dataset_name in ["mlt_Latn", "ind_Latn", "lvs_Latn", "isl_Latn", "khm_Khmr"]:
    dataset_target = pd.read_parquet(
        f"/Users/lujun.li/projects/causallm-github/causalllm/resource/input/ML/datasets/{dataset_name}.parquet"
    ).head(60)
    merged_df = pd.merge(
        eng_ds, dataset_target, on="id", how="inner", suffixes=("_eng", f"_target")
    )[["id", f"sentence_target", "sentence_eng"]]
    merged_df["dataset_name"] = dataset_name
    merged_df_list.append(merged_df)
concatenated_df = pd.concat(merged_df_list, ignore_index=True)

In [9]:
arraycounter = 0
system_message = """You are a translation expert chatbot. Don't answer irrelevant words such as "Of course", just reply to me the translated sentence directly without other unrelated words.
"""
initial_prompt = """
Instruction: {system_message}
Input: 
{INSTRUCTION}
{INPUT}
"""
mixtral_instruct_initial_prompt = """<s>[INST]
{system_message}
{INSTRUCTION}
{INPUT}
[/INST] 
</s>"""

llama2_chat_initial_prompt = """<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{INSTRUCTION}
{INPUT}
[/INST]
"""

llama3_chat_init_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_message}
<|eot_id|><|start_header_id|>user<|end_header_id|>
{INSTRUCTION}
{INPUT}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

code_to_language_dict = {
    "eng_Latn": "English",
    "mlt_Latn": "Maltese",
    "ind_Latn": "Indonesian",
    "lvs_Latn": "Standard Latvian",
    "isl_Latn": "Icelandic",
    "khm_Khmr": "Khmer",
}

In [10]:
from langchain import PromptTemplate

initial_prompt_template = PromptTemplate.from_template(initial_prompt)
llama2_chat_initial_prompt_template = PromptTemplate.from_template(
    llama2_chat_initial_prompt
)
llama3_chat_initial_prompt_template = PromptTemplate.from_template(
    llama3_chat_init_prompt
)
mixtral_instruct_initial_prompt_template = PromptTemplate.from_template(
    mixtral_instruct_initial_prompt
)


def generate_initial_prompts(row):
    language_code = row["dataset_name"]
    language_name = code_to_language_dict[language_code]
    INSTRUCTION = f"Translate the following English sentence to {language_name} and answer me only with the translated sentence."
    INPUT = row["sentence_eng"]
    initial_prompt = initial_prompt_template.format(
        INSTRUCTION=INSTRUCTION, system_message=system_message, INPUT=INPUT
    )
    llama2_chat_initial_prompt = llama2_chat_initial_prompt_template.format(
        INSTRUCTION=INSTRUCTION, system_message=system_message, INPUT=INPUT
    )
    llama3_chat_initial_prompt = llama3_chat_initial_prompt_template.format(
        INSTRUCTION=INSTRUCTION, system_message=system_message, INPUT=INPUT
    )
    mixtral_instruct_initial_prompt = mixtral_instruct_initial_prompt_template.format(
        INSTRUCTION=INSTRUCTION, system_message=system_message, INPUT=INPUT
    )
    row["initial_prompt"] = initial_prompt
    row["llama2_chat_initial_prompt"] = llama2_chat_initial_prompt
    row["llama3_chat_initial_prompt"] = llama3_chat_initial_prompt
    row["mixtral_instruct_initial_prompt"] = mixtral_instruct_initial_prompt
    row["SYSTEM_MESSAGE"] = system_message
    row["USER_MESSAGE"] = INPUT
    return row


concatenated_df = concatenated_df.apply(generate_initial_prompts, axis=1)

In [11]:
temperatures = np.arange(0.1, 2.0, 0.3)
temperature_df = pd.DataFrame({"Temperature": temperatures})
merged_df = temperature_df.merge(concatenated_df, how="cross")

In [12]:
len(merged_df)

2100

In [13]:
merged_df.to_csv("resource/input/ML/ml_experiment_prompt.csv")

In [14]:
exp_df = pd.read_csv("resource/input/ML/ml_experiment_prompt.csv")  ##

exp_df.Temperature.value_counts()

Temperature
0.1    300
0.4    300
0.7    300
1.0    300
1.3    300
1.6    300
1.9    300
Name: count, dtype: int64

In [15]:
exp_df

,Unnamed: 0,Temperature,id,sentence_target,sentence_eng,dataset_name,initial_prompt,llama2_chat_initial_prompt,llama3_chat_initial_prompt,mixtral_instruct_initial_prompt,SYSTEM_MESSAGE,USER_MESSAGE
0,0,0.1,1,"""Issa għandna ġrieden ta' 4 xhur li mhumiex di...","""We now have 4-month-old mice that are non-dia...",mlt_Latn,\nInstruction: You are a translation expert ch...,<s>[INST] <<SYS>>\nYou are a translation exper...,<|begin_of_text|><|start_header_id|>system<|en...,<s>[INST]\nYou are a translation expert chatbo...,You are a translation expert chatbot. Don't an...,"""We now have 4-month-old mice that are non-dia..."
1,1,0.1,2,"Dr. Ehud Ur, professur tal-mediċina fl-Univers...","Dr. Ehud Ur, professor of medicine at Dalhousi...",mlt_Latn,\nInstruction: You are a translation expert ch...,<s>[INST] <<SYS>>\nYou are a translation exper...,<|begin_of_text|><|start_header_id|>system<|en...,<s>[INST]\nYou are a translation expert chatbo...,You are a translation expert chatbot. Don't an...,"Dr. Ehud Ur, professor of medicine at Dalhousi..."
2,2,0.1,3,"Bħal xi esperti oħra, huwa xettiku dwar jekk i...","Like some other experts, he is skeptical about...",mlt_Latn,\nInstruction: You are a translation expert ch...,<s>[INST] <<SYS>>\nYou are a translation exper...,<|begin_of_text|><|start_header_id|>system<|en...,<s>[INST]\nYou are a translation expert chatbo...,You are a translation expert chatbot. Don't an...,"Like some other experts, he is skeptical about..."
3,3,0.1,4,"Nhar it-Tnejn, Sara Danius, segretarja permane...","On Monday, Sara Danius, permanent secretary of...",mlt_Latn,\nInstruction: You are a translation expert ch...,<s>[INST] <<SYS>>\nYou are a translation exper...,<|begin_of_text|><|start_header_id|>system<|en...,<s>[INST]\nYou are a translation expert chatbo...,You are a translation expert chatbot. Don't an...,"On Monday, Sara Danius, permanent secretary of..."
4,4,0.1,5,"Danius qal, ""Bħalissa m’aħna qed nagħmlu xejn....","Danius said, ""Right now we are doing nothing. ...",mlt_Latn,\nInstruction: You are a translation expert ch...,<s>[INST] <<SYS>>\nYou are a translation exper...,<|begin_of_text|><|start_header_id|>system<|en...,<s>[INST]\nYou are a translation expert chatbo...,You are a translation expert chatbot. Don't an...,"Danius said, ""Right now we are doing nothing. ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2095,2095,1.9,56,សមាជិកព្រឹទ្ធសភាបច្ចុប្បន្ន ​និងជាស្រ្តីទីមួយ​...,Current senator and Argentine First Lady Crist...,khm_Khmr,\nInstruction: You are a translation expert ch...,<s>[INST] <<SYS>>\nYou are a translation exper...,<|begin_of_text|><|start_header_id|>system<|en...,<s>[INST]\nYou are a translation expert chatbo...,You are a translation expert chatbot. Don't an...,Current senator and Argentine First Lady Crist...
2096,2096,1.9,57,លោកស្រីឃើចនើរ (Kirchner) បានប្រកាសថា បំណងរបស់ន...,Mrs. Kirchner announced her intention to run f...,khm_Khmr,\nInstruction: You are a translation expert ch...,<s>[INST] <<SYS>>\nYou are a translation exper...,<|begin_of_text|><|start_header_id|>system<|en...,<s>[INST]\nYou are a translation expert chatbo...,You are a translation expert chatbot. Don't an...,Mrs. Kirchner announced her intention to run f...
2097,2097,1.9,58,ការជជែកដេញដោល​នេះត្រូវបានបង្ក​ឡើង​ឱ្យមាន​ភាពចម...,The debate was sparked by controversy over spe...,khm_Khmr,\nInstruction: You are a translation expert ch...,<s>[INST] <<SYS>>\nYou are a translation exper...,<|begin_of_text|><|start_header_id|>system<|en...,<s>[INST]\nYou are a translation expert chatbo...,You are a translation expert chatbot. Don't an...,The debate was sparked by controversy over spe...
2098,2098,1.9,59,ការរិះគន់បែបសេរីលើការប្រឹងប្រែងកសាងឡើងវិញ បានផ...,Liberal criticism of the reconstruction effort...,khm_Khmr,\nInstruction: You are a translation expert ch...,<s>[INST] <<SYS>>\nYou are a translation exper...,<|begin_of_text|><|start_header_id|>system<|en...,<s>[INST]\nYou are a translation expert chatbo...,You are a trans